In [1]:
import sqlite3
import os
from PIL import Image
import io
from pathlib import Path

# Путь к базе данных
DATABASE_PATH = r'C:\work\WestTrade\SecondStep\data\raw_data\so_deep.db'

# Директория для сохранения изображений и меток
OUTPUT_DIR = Path('data/datasets/dataset_ver3')

def mark_id_to_class(wtm_mark_id):
    """
    Преобразует wtm_mark_id в соответствующий класс.
    """
    if 11 <= wtm_mark_id <= 14:
        return 1
    elif 21 <= wtm_mark_id <= 26:
        return 2
    elif 41 <= wtm_mark_id <= 46:
        return 3
    elif wtm_mark_id >= 50:
        return 4
    else:
        return None  # Игнорировать метки вне указанных диапазонов

def main():
    # Создание директорий для классов
    for cls in range(1, 5):
        (OUTPUT_DIR / str(cls)).mkdir(parents=True, exist_ok=True)

    # Подключение к базе данных
    conn = sqlite3.connect(DATABASE_PATH)
    cursor = conn.cursor()

    # Получение списка всех эмбрионов
    cursor.execute("SELECT DISTINCT wtf_ed_uuid FROM well_timeline_frames")
    embryos = [row[0] for row in cursor.fetchall()]

    print(f"Найдено {len(embryos)} эмбрионов.")

    for embryo in embryos:
        print(f"\nОбработка эмбриона: {embryo}")

        # Извлечение всех кадров для текущего эмбриона, отсортированных по номеру серии
        cursor.execute("""
            SELECT wtf_wtl_id, wtf_rel_focus, wtf_frame
            FROM well_timeline_frames
            WHERE wtf_ed_uuid = ?
            ORDER BY wtf_wtl_id ASC
        """, (embryo,))
        frames = cursor.fetchall()

        print(f"  Найдено {len(frames)} кадров.")

        # Извлечение всех меток для текущего эмбриона, отсортированных по номеру серии
        cursor.execute("""
            SELECT wtm_wtl_id, wtm_mark_id, wtm_focus
            FROM well_timeline_marker
            WHERE wtm_ed_uuid = ?
            ORDER BY wtm_wtl_id ASC
        """, (embryo,))
        markers = cursor.fetchall()

        print(f"  Найдено {len(markers)} меток.")

        # Организация меток по фокусу
        markers_by_focus = {}
        for marker in markers:
            wtm_wtl_id, wtm_mark_id, wtm_focus = marker
            if wtm_focus not in markers_by_focus:
                markers_by_focus[wtm_focus] = []
            class_label = mark_id_to_class(wtm_mark_id)
            if class_label is not None:
                markers_by_focus[wtm_focus].append((wtm_wtl_id, class_label))
            else:
                # Добавление метки без класса (для удаления)
                markers_by_focus[wtm_focus].append((wtm_wtl_id, None))

        # Сортировка меток по номеру серии для каждого фокуса
        for focus in markers_by_focus:
            markers_by_focus[focus].sort(key=lambda x: x[0])

        # Обработка каждого кадра
        saved_frames = 0
        for frame in frames:
            wtl_id, rel_focus, frame_blob = frame

            if rel_focus not in markers_by_focus:
                # Нет меток для этого фокуса
                continue

            markers_list = markers_by_focus[rel_focus]

            # Поиск последней метки с wtm_wtl_id <= текущий wtl_id
            class_label = None
            for marker in markers_list:
                marker_wtl_id, marker_class = marker
                if marker_wtl_id > wtl_id:
                    break
                if marker_class is not None:
                    class_label = marker_class
                else:
                    class_label = None  # Удаление метки

            if class_label is not None:
                # Сохранение кадра с соответствующим классом
                try:
                    image = Image.open(io.BytesIO(frame_blob))
                except Exception as e:
                    print(f"    Ошибка при загрузке изображения для кадра {wtl_id} эмбриона {embryo}: {e}")
                    continue

                # Определение пути для сохранения
                class_dir = OUTPUT_DIR / str(class_label)
                embryo_dir = class_dir / embryo
                embryo_dir.mkdir(parents=True, exist_ok=True)

                # Определение имени файла
                image_filename = f"{wtl_id}_{rel_focus}.png"
                image_path = embryo_dir / image_filename

                try:
                    image.save(image_path)
                    saved_frames += 1
                except Exception as e:
                    print(f"    Ошибка при сохранении изображения {image_path}: {e}")

        print(f"  Сохранено {saved_frames} кадров для эмбриона {embryo}.")

    # Закрытие соединения с базой данных
    conn.close()

if __name__ == '__main__':
    main()


Найдено 274 эмбрионов.

Обработка эмбриона: {02C70378-D425-D111-B4A1-A33829BDD99B}
  Найдено 2079 кадров.
  Найдено 13 меток.
  Сохранено 661 кадров для эмбриона {02C70378-D425-D111-B4A1-A33829BDD99B}.

Обработка эмбриона: {03FB4CE8-0C53-241F-45C7-CF22B751E5B6}
  Найдено 1925 кадров.
  Найдено 13 меток.
  Сохранено 477 кадров для эмбриона {03FB4CE8-0C53-241F-45C7-CF22B751E5B6}.

Обработка эмбриона: {078EA5D2-70C3-6F29-3C10-9FFE6E4AE95B}
  Найдено 2037 кадров.
  Найдено 15 меток.
  Сохранено 390 кадров для эмбриона {078EA5D2-70C3-6F29-3C10-9FFE6E4AE95B}.

Обработка эмбриона: {0DEEA966-ACDC-085F-B467-06106A09BAB1}
  Найдено 1713 кадров.
  Найдено 13 меток.
  Сохранено 828 кадров для эмбриона {0DEEA966-ACDC-085F-B467-06106A09BAB1}.

Обработка эмбриона: {0EF5CFBE-71FA-E0CD-9A90-EC42FFFA6A40}
  Найдено 1797 кадров.
  Найдено 16 меток.
  Сохранено 840 кадров для эмбриона {0EF5CFBE-71FA-E0CD-9A90-EC42FFFA6A40}.

Обработка эмбриона: {0F24C158-EB06-9C03-8189-640B95462C4E}
  Найдено 2028 кадров.